In [1]:
# Modify the file 00_setup.py to define input/output file paths on your system
# The information in 00_setup.py will be used across notebooks
from importlib.machinery import SourceFileLoader
setup = SourceFileLoader("setup", "./00_setup.py").load_module()

# ANOVA, Regression, etc.
Various ANOVA and related analyses

In [2]:
import numpy as np
import pandas as pd
from pathlib import Path
import importlib, os

In [3]:
import statsmodels.api as sm
from statsmodels.formula.api import ols
from scipy import stats

In [4]:
import sklearn as sk
sk.__version__

'1.3.2'

## Import Data

In [5]:
sba_loans = pd.read_parquet(Path(setup.temp_path).joinpath('01_DATA_transformed.parquet'))

In [6]:
train_df = sba_loans[sba_loans['dset'] == 'train']

##### NAICS sector (one hot encodings) info

In [7]:
one_hot_feat= [c for c in sba_loans.columns if c.startswith('NS__')]
one_hot_feat_len = len(one_hot_feat)
one_hot_feat_len

10

In [8]:
one_hot_feat

['NS___Accommodation and Food Services',
 'NS___Administrative and Support and Waste Management and Remediation Services',
 'NS___Construction',
 'NS___Health Care and Social Assistance',
 'NS___Manufacturing',
 'NS___Other Services (except Public Administration)',
 'NS___Professional, Scientific, and Technical Services',
 'NS___Retail Trade',
 'NS___Wholesale Trade',
 'NS___infrequent_sklearn']

##### NAICS info

In [9]:
naics_info = pd.read_parquet(Path(setup.temp_path).joinpath('60_DATA_naics_summary_stats.parquet')) 

In [10]:
naics_map = pd.read_parquet(Path(setup.temp_path).joinpath('60_DATA_naics_map.parquet'))

In [11]:
naics_info.sample(2)

,NAICS,train_count,train_target_mean,NAICS_sector,menc_NAICS,cenc_NAICS,dset_naics_holdout,all_count,all_target_mean,NAICS_sector_sel
937,513120,22.0,0.000000,51,0.109988,0.000050,0,29,0.000000,0
263,316992,52.0,0.288462,31-33,0.271208,0.000119,0,74,0.310811,1


##### Neural Network Embeddings
With cluster and NAICS info appended

In [12]:
emb_nn_clus = pd.read_parquet(Path(setup.temp_path).joinpath('60_DATA_embeddings_tsne_naics.parquet'))
emb_nn_feat = [c for c in emb_nn_clus.columns if c.startswith('emb_')]

In [13]:
# Other size clusters, to get correct size
nn_cluster_labels_all = pd.read_parquet(Path(setup.temp_path).joinpath('60_DATA_loop_cluster_labels.parquet'))

##### DGI embeddings

In [14]:
emb_dgi_clus = pd.read_parquet(Path(setup.temp_path).joinpath('63_DATA_embeddings_tsne_naics.parquet'))
emb_dgi_feat = [c for c in emb_dgi_clus.columns if c.startswith('emb_')]

## Functions

##### ANOVA with variation explained

In [15]:
def anova_var(data, eqn):
    """Perform one way ANOVA, return variance explained"""
    lm = ols(eqn, data=data).fit()
    res = sm.stats.anova_lm(lm, typ=1)
    res = pd.concat([res, 
                     (res['sum_sq'].transform(lambda x: x/x.sum())).rename('var_f')],
                    axis=1)
    return res

## NAICS sector - Baseline
ANOVA results in the training data, looking at variation explained by NAICS groupings

##### Stats oneway summary

In [16]:
sector_groups = train_df.groupby('NAICS_sector')
sectors_list = [g['target'].to_numpy() for n, g in sector_groups]

In [17]:
f_sector, p_sector = stats.f_oneway(*sectors_list)

In [18]:
print(f'sector oneway f: {f_sector}, p: {p_sector}')

sector oneway f: 291.0440232558134, p: 0.0


In [19]:
naics_groups = train_df.groupby('NAICS')
naics_list = [g['target'].to_numpy() for n, g in naics_groups]

In [20]:
f_naics, p_naics = stats.f_oneway(*naics_list)

In [21]:
print(f'naics oneway f: {f_naics}, p: {p_naics}')

naics oneway f: 23.185499693712188, p: 0.0


##### Residuals analysis - sector

In [22]:
res = anova_var(train_df, 'target ~ C(NAICS_sector)')

In [23]:
res

,df,sum_sq,mean_sq,F,PR(>F),var_f
C(NAICS_sector),19.0,888.868052,46.782529,291.044023,0.0,0.012521
Residual,436100.0,70098.882943,0.160740,NaN,NaN,0.987479


##### NAICS level 
How much of the mean target by NAICS is accounted for by sector?

In [24]:
naics_info_train = naics_info[naics_info['train_count'] > 0]

In [25]:
anova_var(naics_info_train, 'train_target_mean ~ C(NAICS_sector)')

,df,sum_sq,mean_sq,F,PR(>F),var_f
C(NAICS_sector),19.0,2.092606,0.110137,6.02947,8.001535e-15,0.09088
Residual,1146.0,20.933376,0.018266,NaN,NaN,0.90912


## NN Embeddings

##### NAICS sector ANOVA

In [26]:
# OLS strings for each
emb_str = [ ef+ ' ~ C(NAICS_sector)' for ef in emb_nn_feat]

In [27]:
# All ANOVAs
anova_nn_sector = pd.concat([anova_var(emb_nn_clus, s) for s in emb_str],
                     keys=emb_nn_feat) \
    .reset_index()

In [28]:
anova_nn_sector[anova_nn_sector['level_1'] == 'C(NAICS_sector)']

,level_0,level_1,df,sum_sq,mean_sq,F,PR(>F),var_f
0,emb_000,C(NAICS_sector),19.0,34.749306,1.828911,5.994058,8.483143e-15,0.081065
2,emb_001,C(NAICS_sector),19.0,43.070598,2.266874,5.581172,1.823633e-13,0.075905
4,emb_002,C(NAICS_sector),19.0,14.829540,0.780502,3.499314,5.547648e-07,0.048978
6,emb_003,C(NAICS_sector),19.0,4.942862,0.260151,2.981435,1.749886e-05,0.042034
8,emb_004,C(NAICS_sector),19.0,17.984602,0.946558,4.705865,1.115476e-10,0.064772
10,emb_005,C(NAICS_sector),19.0,4.785227,0.251854,2.492737,3.791076e-04,0.035388
12,emb_006,C(NAICS_sector),19.0,14.099265,0.742067,4.576102,2.849018e-10,0.063098
14,emb_007,C(NAICS_sector),19.0,38.176746,2.009302,5.105779,6.053271e-12,0.069891


##### NAICS level ANOVA (mean target)

In [29]:
naics_info_train_nn_cluster = naics_info[naics_info['train_count'] > 0] \
    .merge(emb_nn_clus.drop(columns='NAICS').rename(columns={'NAICS_orig':'NAICS'})[['NAICS', 'cluster']],
           how='left', on='NAICS')

In [30]:
anova_var(naics_info_train_nn_cluster, 'train_target_mean ~ C(cluster)')

,df,sum_sq,mean_sq,F,PR(>F),var_f
C(cluster),8.0,13.589351,1.698669,208.269227,5.202043e-218,0.590175
Residual,1157.0,9.436631,0.008156,NaN,NaN,0.409825


##### Loan level ANOVA (raw target)

In [31]:
train_nn_cluster = train_df \
    .merge(emb_nn_clus.drop(columns='NAICS').rename(columns={'NAICS_orig':'NAICS'})[['NAICS', 'cluster']],
           how='left', on='NAICS')

In [32]:
anova_var(train_nn_cluster, 'target ~ C(cluster)')

,df,sum_sq,mean_sq,F,PR(>F),var_f
C(cluster),8.0,3122.808761,390.351095,2508.458725,0.0,0.043991
Residual,436111.0,67864.942234,0.155614,NaN,NaN,0.956009


## DGI Embeddings

##### NAICS sector ANOVA

In [33]:
# OLS strings for each
emb_dgi_str = [ ef+ ' ~ C(NAICS_sector)' for ef in emb_dgi_feat]

In [34]:
# All ANOVAs
anova_dgi_sector = pd.concat([anova_var(emb_dgi_clus, s) for s in emb_dgi_str],
                     keys=emb_nn_feat) \
    .reset_index()

In [35]:
anova_dgi_sector[anova_nn_sector['level_1'] == 'C(NAICS_sector)']

,level_0,level_1,df,sum_sq,mean_sq,F,PR(>F),var_f
0,emb_000,C(NAICS_sector),19.0,0.108158,5.692513e-03,11.683082,2.016715e-33,0.146716
2,emb_001,C(NAICS_sector),19.0,0.147171,7.745857e-03,11.683083,2.016699e-33,0.146716
4,emb_002,C(NAICS_sector),19.0,0.000003,1.341157e-07,11.683084,2.016687e-33,0.146716
6,emb_003,C(NAICS_sector),19.0,0.228641,1.203372e-02,11.683083,2.016693e-33,0.146716
8,emb_004,C(NAICS_sector),19.0,6.516217,3.429588e-01,3.803089,6.849291e-08,0.053004
10,emb_005,C(NAICS_sector),19.0,27.964960,1.471840e+00,10.417260,2.734344e-29,0.132933
12,emb_006,C(NAICS_sector),19.0,0.546158,2.874516e-02,11.173232,9.245548e-32,0.141218
14,emb_007,C(NAICS_sector),19.0,47.372045,2.493266e+00,30.928318,2.478311e-91,0.312800


##### NAICS level ANOVA (mean target)

In [36]:
naics_info_train_dgi_cluster = naics_info[naics_info['train_count'] > 0] \
    .merge(emb_dgi_clus.rename(columns={'NAICS_orig':'NAICS'})[['NAICS', 'cluster']],
           how='left', on='NAICS')

In [37]:
anova_var(naics_info_train_dgi_cluster, 'train_target_mean ~ C(cluster)')

,df,sum_sq,mean_sq,F,PR(>F),var_f
C(cluster),9.0,5.017097,0.557455,35.783352,4.069852e-56,0.217889
Residual,1156.0,18.008885,0.015579,NaN,NaN,0.782111


##### Loan level ANOVA (raw target)

In [38]:
train_dgi_cluster = train_df \
    .merge(emb_dgi_clus.rename(columns={'NAICS_orig':'NAICS'})[['NAICS', 'cluster']],
           how='left', on='NAICS')

In [39]:
anova_var(train_dgi_cluster, 'target ~ C(cluster)')

,df,sum_sq,mean_sq,F,PR(>F),var_f
C(cluster),9.0,1778.427880,197.603098,1245.16009,0.0,0.025053
Residual,436110.0,69209.323115,0.158697,NaN,NaN,0.974947


## Logistic Regression
Very simple models to analyze information in high-level groupings (Sector and from embeddings)

Use same-size one hot features for each group, train data only.

##### Get fixed-level NN clusters 
The NN clusters are only for training; get one_hot_feat_len of these (unlike earlier scripts where I used 

In [40]:
sel_labels_nn = emb_nn_clus[['NAICS_orig']] \
    .merge(nn_cluster_labels_all[nn_cluster_labels_all['n_clusters'] == one_hot_feat_len] \
               .drop(columns='n_clusters'),
           left_index=True, right_index = True)
sel_labels_nn.shape

(1166, 2)

In [41]:
# DGI clusters - limit to training match
emb_dgi_train = emb_dgi_clus \
    .rename(columns={'cluster':'cluster_dgi','NAICS_orig':'NAICS'}) \
    .merge(train_df[['NAICS']].drop_duplicates(), on='NAICS') 

In [42]:
all_naics_clus = emb_dgi_train \
    .merge(sel_labels_nn.rename(columns={'NAICS_orig':'NAICS', 'label':'cluster_nn'}), on='NAICS')
all_naics_clus[['cluster_dgi', 'cluster_nn']] = all_naics_clus[['cluster_dgi', 'cluster_nn']] \
    .apply(lambda x: x.astype('category'))

In [43]:
lm_df = train_df[['LoanNr_ChkDgt', 'NAICS','dset', 'NAICS_sector', 'target']] \
     .merge(all_naics_clus[['NAICS', 'cluster_dgi', 'cluster_nn']],
                          how='left', on='NAICS') \
    .merge(naics_info[['NAICS', 'NAICS_sector_sel']], how='left', on='NAICS')

In [44]:
lm_df['NAICS_sector_sel'] = np.where(lm_df['NAICS_sector_sel'] == 1,lm_df['NAICS_sector'],
                                     'other')
#lm_df['NAICS_sector_sel'] = lm_df['NAICS_sector_sel'].astype('category')

##### Function to do the logistic regression

In [45]:
def log_reg(data, feature = ['NAICS_sector_sel']):
    X = pd.get_dummies(data[feature], dtype='float', drop_first=True)
    Y = data['target']
    Xi = sm.add_constant(X)
    sm_logreg = sm.Logit(Y, Xi)

    res = sm_logreg.fit_regularized(method='l1')

    return res
                            

##### Single group models

In [46]:
res_naics_sector = log_reg(lm_df)
print(res_naics_sector.summary())

Optimization terminated successfully    (Exit mode 0)
            Current function value: 0.5014901419293476
            Iterations: 74
            Function evaluations: 75
            Gradient evaluations: 74
                           Logit Regression Results                           
Dep. Variable:                 target   No. Observations:               436120
Model:                          Logit   Df Residuals:                   436110
Method:                           MLE   Df Model:                            9
Date:                Sun, 07 Apr 2024   Pseudo R-squ.:                 0.01045
Time:                        21:42:38   Log-Likelihood:            -2.1871e+05
converged:                       True   LL-Null:                   -2.2102e+05
Covariance Type:            nonrobust   LLR p-value:                     0.000
                             coef    std err          z      P>|z|      [0.025      0.975]
-------------------------------------------------------------------

In [47]:
res_cluster_dgi = log_reg(lm_df, ['cluster_dgi'])
print(res_cluster_dgi.summary())

Optimization terminated successfully    (Exit mode 0)
            Current function value: 0.4932504837515452
            Iterations: 70
            Function evaluations: 71
            Gradient evaluations: 70
                           Logit Regression Results                           
Dep. Variable:                 target   No. Observations:               436120
Model:                          Logit   Df Residuals:                   436110
Method:                           MLE   Df Model:                            9
Date:                Sun, 07 Apr 2024   Pseudo R-squ.:                 0.02671
Time:                        21:42:42   Log-Likelihood:            -2.1512e+05
converged:                       True   LL-Null:                   -2.2102e+05
Covariance Type:            nonrobust   LLR p-value:                     0.000
                    coef    std err          z      P>|z|      [0.025      0.975]
----------------------------------------------------------------------------

In [48]:
res_cluster_nn = log_reg(lm_df, ['cluster_nn'])
print(res_cluster_nn.summary())

Optimization terminated successfully    (Exit mode 0)
            Current function value: 0.4832466742488224
            Iterations: 80
            Function evaluations: 81
            Gradient evaluations: 80
                           Logit Regression Results                           
Dep. Variable:                 target   No. Observations:               436120
Model:                          Logit   Df Residuals:                   436110
Method:                           MLE   Df Model:                            9
Date:                Sun, 07 Apr 2024   Pseudo R-squ.:                 0.04645
Time:                        21:42:47   Log-Likelihood:            -2.1075e+05
converged:                       True   LL-Null:                   -2.2102e+05
Covariance Type:            nonrobust   LLR p-value:                     0.000
                   coef    std err          z      P>|z|      [0.025      0.975]
-----------------------------------------------------------------------------

##### Two-way models

In [49]:
res_2_sector_nn = log_reg(lm_df, ['NAICS_sector_sel', 'cluster_nn'])
print(res_2_sector_nn.summary())

Optimization terminated successfully    (Exit mode 0)
            Current function value: 0.4818791787309131
            Iterations: 137
            Function evaluations: 137
            Gradient evaluations: 137
                           Logit Regression Results                           
Dep. Variable:                 target   No. Observations:               436120
Model:                          Logit   Df Residuals:                   436101
Method:                           MLE   Df Model:                           18
Date:                Sun, 07 Apr 2024   Pseudo R-squ.:                 0.04915
Time:                        21:42:56   Log-Likelihood:            -2.1016e+05
converged:                       True   LL-Null:                   -2.2102e+05
Covariance Type:            nonrobust   LLR p-value:                     0.000
                             coef    std err          z      P>|z|      [0.025      0.975]
----------------------------------------------------------------

In [50]:
res_2_sector_dgi = log_reg(lm_df, ['NAICS_sector_sel', 'cluster_dgi'])
print(res_2_sector_dgi.summary())

Optimization terminated successfully    (Exit mode 0)
            Current function value: 0.49066527975408875
            Iterations: 121
            Function evaluations: 121
            Gradient evaluations: 121
                           Logit Regression Results                           
Dep. Variable:                 target   No. Observations:               436120
Model:                          Logit   Df Residuals:                   436101
Method:                           MLE   Df Model:                           18
Date:                Sun, 07 Apr 2024   Pseudo R-squ.:                 0.03181
Time:                        21:43:04   Log-Likelihood:            -2.1399e+05
converged:                       True   LL-Null:                   -2.2102e+05
Covariance Type:            nonrobust   LLR p-value:                     0.000
                             coef    std err          z      P>|z|      [0.025      0.975]
---------------------------------------------------------------

In [51]:
res_2_nn_dgi = log_reg(lm_df, ['cluster_nn', 'cluster_dgi'])
print(res_2_nn_dgi.summary())

Optimization terminated successfully    (Exit mode 0)
            Current function value: 0.4804591126440137
            Iterations: 145
            Function evaluations: 145
            Gradient evaluations: 145
                           Logit Regression Results                           
Dep. Variable:                 target   No. Observations:               436120
Model:                          Logit   Df Residuals:                   436101
Method:                           MLE   Df Model:                           18
Date:                Sun, 07 Apr 2024   Pseudo R-squ.:                 0.05195
Time:                        21:43:10   Log-Likelihood:            -2.0954e+05
converged:                       True   LL-Null:                   -2.2102e+05
Covariance Type:            nonrobust   LLR p-value:                     0.000
                    coef    std err          z      P>|z|      [0.025      0.975]
-------------------------------------------------------------------------

##### 3 Way

In [52]:
res_3 = log_reg(lm_df, ['cluster_nn', 'cluster_dgi', 'NAICS_sector_sel'])
print(res_3.summary())

Optimization terminated successfully    (Exit mode 0)
            Current function value: 0.4794015244656354
            Iterations: 184
            Function evaluations: 184
            Gradient evaluations: 184
                           Logit Regression Results                           
Dep. Variable:                 target   No. Observations:               436120
Model:                          Logit   Df Residuals:                   436092
Method:                           MLE   Df Model:                           27
Date:                Sun, 07 Apr 2024   Pseudo R-squ.:                 0.05404
Time:                        21:43:18   Log-Likelihood:            -2.0908e+05
converged:                       True   LL-Null:                   -2.2102e+05
Covariance Type:            nonrobust   LLR p-value:                     0.000
                             coef    std err          z      P>|z|      [0.025      0.975]
----------------------------------------------------------------

##### Gather Metrics

In [53]:
def get_metric_ser(res):
    return pd.Series([res.aic, res.bic, res.prsquared],
                     index=['aic', 'bic', 'prsquared'])

In [54]:
res_list = [res_naics_sector, res_cluster_dgi, res_cluster_nn, 
            res_2_sector_dgi, res_2_sector_nn, res_2_nn_dgi, res_3]
res_name_list = ['sector', 'dgi', 'nn',  'sector + dgi', 'sector + nn',
                 'dgi + nn', 'sector + dgi + nn']
logit_metrics = pd.concat([get_metric_ser(x).to_frame().transpose() for x in res_list],
                          keys=res_name_list, axis=0) \
    .reset_index(level=1, drop=True) \
    .reset_index() \
    .rename(columns={'index':'model'})
logit_metrics

,model,aic,bic,prsquared
0,sector,437439.761396,437549.618124,0.010454
1,dgi,430252.801947,430362.658675,0.026712
2,nn,421527.079147,421636.935874,0.046452
3,sector + dgi,428015.883613,428224.611394,0.031813
4,sector + nn,420352.294856,420561.022638,0.049150
5,dgi + nn,419113.656413,419322.384194,0.051952
6,sector + dgi + nn,418209.185700,418516.784536,0.054039


In [55]:
logit_metrics.to_csv(Path(setup.temp_path).joinpath('81_REPORT_logit_metrics.csv'), index=False)